In [ ]:
%pip install clarifai-pyspark

# Testing ClarifaiPySpark SDK

### Getting Started

In [ ]:
from clarifaipyspark.client import ClarifaiPySpark
from pyspark.sql import SparkSession

CLARIFAI_PAT = dbutils.secrets.get(scope="clarifai", key="clarifai-pat")

2023-11-17 10:47:29 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=690049;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=59966;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Creating ClarifaiPyspark object & creating/fetching image dataset from app

In [ ]:
cspark_obj = ClarifaiPySpark(user_id='mansi_k', app_id='databricks_tester_img', pat=CLARIFAI_PAT)

dataset_obj = cspark_obj.dataset(dataset_id='dataset1')

2023-11-17 10:37:00 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=424539;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=746841;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Fetching current inputs from image dataset

In [ ]:
my_inputs = list(dataset_obj.list_inputs(input_type='image'))
my_inputs

2023-11-17 10:39:52 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=653630;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=683346;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

[id: "c10"
 data {
   image {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/databricks_tester_img/inputs/image/c1b5a9392f89cf42deca348758a63c79"
     hosted {
       prefix: "https://data.clarifai.com"
       suffix: "users/mansi_k/apps/databricks_tester_img/inputs/image/c1b5a9392f89cf42deca348758a63c79"
       sizes: "orig"
       sizes: "tiny"
       sizes: "small"
       sizes: "large"
       crossorigin: "use-credentials"
     }
     image_info {
       width: 1000
       height: 665
       format: "JPEG"
       color_mode: "YUV"
     }
   }
   concepts {
     id: "id-cat"
     name: "cat"
     value: 1.0
     app_id: "databricks_tester_img"
   }
 }
 created_at {
   seconds: 1698049660
   nanos: 961070000
 }
 modified_at {
   seconds: 1698049664
   nanos: 89517000
 }
 status {
   code: INPUT_DOWNLOAD_SUCCESS
   description: "Download complete"
 },
 id: "c3"
 data {
   image {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/databricks_tester_img/inputs/im

### Upload images from volume folder to Clarifai dataset

In [ ]:
dataset_obj.upload_dataset_from_folder(folder_path='/Volumes/mansi_test/default/cat', input_type='image', labels=True)

2023-10-23 08:27:37 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=55296;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=786623;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:05<00:00,  5.36s/it]


### Export image annotations to spark dataframe

In [ ]:
annot_df = dataset_obj.export_annotations_to_dataframe()
annot_df

2023-11-17 10:42:14 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=867812;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=60747;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

DataFrame[annotation: string, annotation_created_at: string, annotation_id: string, annotation_modified_at: string, annotation_user_id: string, input_id: string]

### Write annotations dataframe to volume as delta table

In [ ]:
annot_df.write.format("delta").mode("overwrite").save("/Volumes/mansi_test/default/test_vol1/imgAnnsDeltaTable2")


2023-11-17 10:43:04 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=804695;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=346850;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Load annotations delta table from volume

In [ ]:
spark = SparkSession.builder.appName("Clarifai-pyspark").getOrCreate()
spark.conf.set("spark.databricks.agent.id", "clarifaipyspark")
df_delta = spark.read.format("delta").load("/Volumes/mansi_test/default/test_vol1/imgAnnsDeltaTable2")
df_delta.show()

2023-11-17 10:45:04 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=912153;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=905134;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+--------------------+---------------------+--------------------+----------------------+------------------+--------+
|          annotation|annotation_created_at|       annotation_id|annotation_modified_at|annotation_user_id|input_id|
+--------------------+---------------------+--------------------+----------------------+------------------+--------+
|concepts {\n  id:...|  10/23/% 08:27:   %5|b3847d13486f4f118...|   10/23/% 08:27:   %5|           mansi_k|      c2|
|concepts {\n  id:...|  10/23/% 08:27:   %5|f65d75b3446b40ffa...|   10/23/% 08:27:   %5|           mansi_k|     c11|
|concepts {\n  id:...|  11/17/% 10:41:   %5|76ac2fcb998440bc8...|   11/17/% 10:41:   %5|           mansi_k|      c8|
|concepts {\n  id:...|  11/17/% 10:41:   %5|7edf0bc509454a9ca...|   11/17/% 10:41:   %5|           mansi_k|      c4|
|concepts {\n  id:...|  10/23/% 08:27:   %5|8fd99dc05eef46849...|   10/23/% 08:27:   %5|           mansi_k|      c1|
|concepts {\n  id:...|  11/17/% 10:41:   %5|f044bc745f754544b...

### Loop through delta table rows & columns

In [ ]:
for row in df_delta.collect()[:4]:
    print(row['annotation_id'])
    print(row['input_id'])
    print(row['annotation_user_id'])
    print(row['annotation_created_at'])
    print(row['annotation_modified_at'])
    print(row['annotation'])
    print("########################\n")

2023-11-17 10:46:19 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=745449;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=380513;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

b3847d13486f4f1189315ad4565221db
c2
mansi_k
10/23/% 08:27:   %5
10/23/% 08:27:   %5
concepts {
  id: "id-cat"
  name: "cat"
  value: 1.0
  app_id: "databricks_tester_img"
}

########################

f65d75b3446b40ffa012735a31dcbb62
c11
mansi_k
10/23/% 08:27:   %5
10/23/% 08:27:   %5
concepts {
  id: "id-cat"
  name: "cat"
  value: 1.0
  app_id: "databricks_tester_img"
}

########################

76ac2fcb998440bc8d2a5fffd3a8ae1a
c8
mansi_k
11/17/% 10:41:   %5
11/17/% 10:41:   %5
concepts {
  id: "id-cat"
  name: "cat"
  value: 1.0
  app_id: "databricks_tester_img"
}

########################

7edf0bc509454a9caa74ac15d5604211
c4
mansi_k
11/17/% 10:41:   %5
11/17/% 10:41:   %5
concepts {
  id: "id-cat"
  name: "cat"
  value: 1.0
  app_id: "databricks_tester_img"
}

########################



### Testing export_images_to_volume()

In [ ]:
dataset_obj.export_images_to_volume(path="/Volumes/mansi_test/default/test_vol1", input_response=my_inputs[:5])

2023-10-23 08:31:44 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=244234;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=125648;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Load image csv file & upload text data into app using dataframe

In [ ]:
spark = SparkSession.builder.appName("Clarifai-pyspark").getOrCreate()
spark.conf.set("spark.databricks.agent.id", "clarifaipyspark")
df = spark.read.option("header",True).csv("/Volumes/mansi_test/default/test_vol1/img_data2.csv")
df.show()

2023-11-17 10:47:41 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=757081;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=883448;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+-------+--------------------+--------+
|inputid|               input|concepts|
+-------+--------------------+--------+
|  img11|https://img.freep...|     dog|
|  img21|https://images.un...|    duck|
|  img31|https://mymodernm...|    duck|
+-------+--------------------+--------+



In [ ]:
dataset_obj.upload_dataset_from_dataframe(dataframe=df, input_type='image', df_type='url', labels=True)

2023-11-17 10:48:02 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=167577;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=322666;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:02<00:00,  2.49s/it]


### Using a custom dataloader to upload images from Volume

In [ ]:
# Form a dataloader
from clarifai.datasets.upload.utils import load_module_dataloader

food_dataloader = load_module_dataloader(module_dir="/Volumes/mansi_test/default/test_vol1/food-101")

In [ ]:
dataset_obj.upload_dataset_from_dataloader(dataloader=food_dataloader)

2023-11-17 10:48:29 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=454243;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=256824;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading Dataset: 100%|██████████| 1/1 [00:02<00:00,  2.63s/it]


### Creating ClarifaiPyspark object & creating/fetching text dataset from app

In [ ]:
cspark_obj = ClarifaiPySpark(user_id='mansi_k', app_id='databricks_tester_txt', pat=CLARIFAI_PAT)

dataset_obj = cspark_obj.dataset(dataset_id='dataset1')

2023-11-17 10:48:46 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=496906;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=172685;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Fetching current inputs from text dataset

In [ ]:
my_inputs = list(dataset_obj.list_inputs(input_type='text'))

my_inputs

2023-11-17 10:49:23 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=658476;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=961521;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

[id: "XFmGD0xHlNXgGIXF"
 data {
   text {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/databricks_tester_txt/inputs/text/65c1aa5487711a23f7477200fd01e253"
     hosted {
       prefix: "https://data.clarifai.com"
       suffix: "users/mansi_k/apps/databricks_tester_txt/inputs/text/65c1aa5487711a23f7477200fd01e253"
       sizes: "orig"
       crossorigin: "use-credentials"
     }
     text_info {
       char_count: 3
       encoding: "UTF8"
     }
   }
 }
 created_at {
   seconds: 1698678263
   nanos: 736342000
 }
 modified_at {
   seconds: 1698678273
   nanos: 49546000
 }
 status {
   code: INPUT_DOWNLOAD_SUCCESS
   description: "Download complete"
 },
 id: "Ak1n8DZ1l1RWKATv"
 data {
   text {
     url: "https://data.clarifai.com/orig/users/mansi_k/apps/databricks_tester_txt/inputs/text/5a8dd3ad0756a93ded72b823b19dd877"
     hosted {
       prefix: "https://data.clarifai.com"
       suffix: "users/mansi_k/apps/databricks_tester_txt/inputs/text/5a8dd3ad0756a93ded72b823b19

### Testing export_text_to_volume()

Q- Create folders acc to concept name?   
Q- Save text as csv?

In [ ]:
dataset_obj.export_text_to_volume(path="/Volumes/mansi_test/default/test_vol1", input_response=my_inputs)

2023-11-17 10:49:40 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=780975;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=579939;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Upload text from volume csv to Clarifai dataset

In [ ]:
dataset_obj.upload_dataset_from_csv(csv_path='/Volumes/mansi_test/default/test_vol1/emotions_data1.csv', input_type='text', labels=True, csv_type='raw')

2023-10-23 08:38:10 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=990114;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=279494;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs: 100%|██████████| 1/1 [00:20<00:00, 20.56s/it]


### Load test csv file & upload text data into app using dataframe

In [ ]:
spark = SparkSession.builder.appName("Clarifai-pyspark").getOrCreate()
spark.conf.set("spark.databricks.agent.id", "clarifaipyspark")
df = spark.read.option("header",True).csv("/Volumes/mansi_test/default/test_vol1/emotions_data1.csv")
df.show()

2023-11-17 10:50:18 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=677723;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=27871;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+-------+--------------------+--------+
|inputid|               input|concepts|
+-------+--------------------+--------+
|    t11|   I am feeling good|   happy|
|    t21|This is a guava tree| neutral|
|    t31|This is a saddeni...|     sad|
+-------+--------------------+--------+



In [ ]:
dataset_obj.upload_dataset_from_dataframe(dataframe=df, input_type='text', df_type='raw', labels=True)

2023-10-23 08:39:47 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=241980;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=257321;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

Uploading inputs:   0%|          | 0/1 [00:00<?, ?it/s]

2023-10-23 08:39:48 WARNING  WARNING:clarifai.client.input:code: INPUT_INVALID_ARGUMENT                ]8;id=620274;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py\input.py]8;;\:]8;id=60565;file:///local_disk0/.ephemeral_nfs/envs/pythonEnv-07884fc6-4bdc-4cb9-bce0-6f0de44f7080/lib/python3.10/site-packages/clarifai/client/input.py#631\631]8;;\
                             description: "Input invalid argument"                                                 
                             details: "Input has a duplicate ID."                                                  
                                                                                                                   

Uploading inputs: 100%|██████████| 1/1 [00:00<00:00,  5.45it/s]
